In [ ]:
# Gros exemple

conn = sqlite3.connect("ma_base.db")
cursor = conn.cursor()
cursor.execute("CREATE TABLE movie(title, year, score)")

res = cursor.execute("SELECT name FROM sqlite_master")
res.fetchall()

cursor.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

conn.commit() # Permet de valider toutes les requêtes du style INSERT, UPDATE, DELETE etc.
# tous ce qui est modif de la bdd

res = cursor.execute("SELECT score FROM movie")
res.fetchall()

data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cursor.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
conn.commit()  # Remember to commit the transaction after executing INSERT.

# Deux méthodes pour afficher les lignes
cursor.execute("SELECT year, title FROM movie ORDER BY year").fetchall() # OU
for row in cursor.execute("SELECT year, title FROM movie ORDER BY year"):
    print(row)

conn.close()


In [ ]:
#Exemple chatgpt

cursor = conn.cursor()

# Création d'une table exemple
cursor.execute("""
CREATE TABLE IF NOT EXISTS utilisateurs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nom TEXT,
    age INTEGER
)
""")

# Insertion de données
cursor.execute("INSERT INTO utilisateurs (nom, age) VALUES (?, ?)", ("Alice", 30))
cursor.execute("INSERT INTO utilisateurs (nom, age) VALUES (?, ?)", ("Bob", 25))

conn.commit()
conn.close()

In [1]:
import sqlite3
import pandas as pd
import requests

In [2]:
conn = sqlite3.connect("data/test.db")
curs = conn.cursor()

In [3]:
curs.execute("""
CREATE TABLE IF NOT EXISTS produits (
    id_produit TEXT PRIMARY KEY,
    nom TEXT,
    prix REAL,
    stock INTEGER
);
""")

In [4]:
val = pd.read_sql("SELECT id_produit FROM produits", conn)
val

,id_produit


In [ ]:
type(val['id_produit'])

In [ ]:
# Conditions
prod_sql = pd.read_sql_table("produits", "data/test.db")
mags_sql = pd.read_sql_table("magasins", "data/test.db")
ventes_sql = pd.read_sql_table("ventes", "data/test.db")

In [ ]:
# Import des packages
import sqlite3
import pandas as pd
import requests

# Création de la base en format db + créations des 3 tables
conn = sqlite3.connect("ma_base.db")
curs = conn.cursor()

curs.execute("""
CREATE TABLE IF NOT EXISTS produits (
    id_produit TEXT PRIMARY KEY,
    nom TEXT,
    prix REAL,
    stock INTEGER
);
""")

curs.execute("""
CREATE TABLE IF NOT EXISTS magasins (
    id_magasin INTEGER PRIMARY KEY,
    ville TEXT,
    nb_salaries INTEGER
);
""")

curs.execute("""
CREATE TABLE IF NOT EXISTS ventes (
    id_vente INTEGER PRIMARY KEY,
    date TEXT,
    id_produit INTEGER,
    quantite INTEGER,
    id_magasin INTEGER,
    FOREIGN KEY (id_produit) REFERENCES produits(id_produit),
    FOREIGN KEY (id_magasin) REFERENCES magasins(id_magasin)
);
""")

# Chargement des données en csv
dico={'ventes' : "https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=760830694&single=true&output=csv",
      'produits' : "https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=0&single=true&output=csv",
      'magasins' : "https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=714623615&single=true&output=csv"}

for nom, url in dico.items():
    reponse=requests.get(url)
    data=reponse.text
    with open(nom+".csv", 'w') as f:
        f.write(data)
        f.close()
    
# renommage des colonnes et correction des types de données
prod = pd.read_csv("produits.csv")
mags = pd.read_csv("magasins.csv")
ventes = pd.read_csv("ventes.csv")
ventes["ID_vente"]=range(1,ventes.shape[0]+1)

prod.rename(columns={"Nom" : "nom",
             "ID RÃ©fÃ©rence produit" : "id_produit",
             "Prix" : "prix",
             "Stock":"stock"}, inplace=True)

mags.rename(columns={"ID Magasin" : "id_magasin",
             "Ville" : "ville",
             "Nombre de salariÃ©s" : "nb_salaries"}, inplace=True)

ventes.rename(columns={"Date" : "date",
               "ID RÃ©fÃ©rence produit" : "id_produit",
               "QuantitÃ©" : "quantite",
               "ID Magasin" : "id_magasin",
               "ID_vente" : "id_vente"}, inplace=True)

# Insertion des valeurs dans les tables à partir des pd.DataFrame 
prod.to_sql("produits", conn, if_exists="append", index=False)
mags.to_sql("magasins", conn, if_exists="append", index=False)
ventes.to_sql("ventes", conn, if_exists="append", index=False)

conn.commit()
conn.close()